In [6]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta

In [7]:
def dict_creator(
    publication_date, company, job_title, position, website_name, link_url
):
    offers_dict = dict()
    offers_dict["publication_date"] = publication_date
    offers_dict["company"] = company
    offers_dict["title"] = job_title
    offers_dict["position"] = position
    offers_dict["website"] = website_name
    offers_dict["link_url"] = link_url

    return offers_dict

In [20]:
def get_nofluffjobs_job_details(job_element):
    company_element = job_element.find(
        "span", class_="d-block posting-title__company text-truncate"
    )
    company = company_element.text.strip()
    job_title_element = job_element.find(
        "h3",
        class_="posting-title__position text-truncate color-main ng-star-inserted",
    )
    job_title = job_title_element.text.strip()

    link_url = "https://nofluffjobs.com" + job_element["href"]
    pattern = re.compile(r"https?://([\w.\.\-]+)")
    website_name = pattern.match(link_url)[0]
    page_job_element = requests.get(link_url)
    soup_page = BeautifulSoup(page_job_element.content, "html.parser")
    position = soup_page.find("span", class_="mr-10 font-weight-medium").text.strip()
    publication_date_element = soup_page.select('div[class*="tw-h-12 "]')
    print(job_title)
    print(publication_date_element)
    print(position)
    """days_after_publication = re.findall(
        r"\b\d+\b", publication_date_element.text.strip()
    )
    
    if not days_after_publication:
        publication_date = datetime.today().strftime("%Y-%m-%d")


    else:
        publication_date = (
            datetime.today() - (timedelta(days=int(days_after_publication[0])))
        ).strftime("%Y-%m-%d")"""
    publication_date = "2020-10-24"
    return (publication_date, company, job_title, position, website_name, link_url)


In [9]:
def nofluffjobs_page_job_offers(
    url="https://nofluffjobs.com/pl/praca-zdalna/python?criteria=city%3Dwarszawa%20seniority%3Dtrainee,junior%20%20salary<pln12000m&page=1",
) -> list:

    nofluffjobs_list = list()

    flag = True
    try:
        page = requests.get(url)
    except requests.exceptions.ConnectionError as err:
        flag = False
    except requests.exceptions.MissingSchema as err:
        flag = False
    if flag:
        soup = BeautifulSoup(page.content, "html.parser")
        job_elements = soup.select('a[class*="posting-list-item posting-list-item--"]')
        for job_element in job_elements:

            (
                publication_date,
                company,
                job_title,
                position,
                website_name,
                link_url,
            ) = get_nofluffjobs_job_details(job_element)
            nofluffjobs_dict = dict_creator(
                publication_date, company, job_title, position, website_name, link_url
            )
            nofluffjobs_list.append(nofluffjobs_dict)
    return nofluffjobs_list


In [21]:
import pandas as pd
import numpy as np

nofluffjobs_list = nofluffjobs_page_job_offers()
df_raw = pd.DataFrame.from_records(nofluffjobs_list)
df = df_raw.copy()
df['publication_date'] = pd.to_datetime(df['publication_date'], infer_datetime_format=True)
df.drop_duplicates(subset=['publication_date', 'company', 'title'], inplace=True, ignore_index=True)




Junior Python Data Developer
[]
Junior
Akademia Cloud Data Engineer
[]
Trainee, Junior
Data Engineer with Python
[]
Junior
Junior Python Developer
[]
Junior
Junior Python Developer
[]
Junior
Junior Data Engineer
[]
Junior
Junior Data Engineer
[]
Junior
Junior Full Stack Developer Remote
[]
Junior
Junior Full Stack Developer Remote
[]
Junior
Remote Junior Python Developer
[]
Junior
Remote Junior DevOps
[]
Junior
Remote Junior DevOps
[]
Junior
Junior Data Scientist
[]
Junior
Junior Fullstack Developer
[]
Junior
Junior Fullstack Developer
[]
Junior
Junior Data Engineer
[]
Junior
Junior C++ Developer
[]
Junior
Junior C++ Developer
[]
Junior


In [17]:
df

,publication_date,company,title,position,website,link_url
0,2020-10-24,Onwelo S.A.,Junior Python Data Developer,Junior,https://nofluffjobs.com,https://nofluffjobs.com/pl/job/junior-python-d...
1,2020-10-24,Onwelo S.A.,Akademia Cloud Data Engineer,"Trainee, Junior",https://nofluffjobs.com,https://nofluffjobs.com/pl/job/akademia-cloud-...
2,2020-10-24,AXA Avanssur SA Spółka Akcyjna Oddział II w Po...,Data Engineer with Python,Junior,https://nofluffjobs.com,https://nofluffjobs.com/pl/job/data-engineer-w...
3,2020-10-24,Green Minds Sp.z o.o.,Junior Python Developer,Junior,https://nofluffjobs.com,https://nofluffjobs.com/pl/job/junior-python-d...
4,2020-10-24,Addepto,Junior Data Engineer,Junior,https://nofluffjobs.com,https://nofluffjobs.com/pl/job/junior-data-eng...
5,2020-10-24,ImpiCode,Junior Full Stack Developer Remote,Junior,https://nofluffjobs.com,https://nofluffjobs.com/pl/job/junior-full-sta...
6,2020-10-24,More Code Sp. z o.o.,Remote Junior Python Developer,Junior,https://nofluffjobs.com,https://nofluffjobs.com/pl/job/remote-junior-p...
7,2020-10-24,Antal Poland,Remote Junior DevOps,Junior,https://nofluffjobs.com,https://nofluffjobs.com/pl/job/remote-junior-d...
8,2020-10-24,Simon - Kucher & Partners,Junior Data Scientist,Junior,https://nofluffjobs.com,https://nofluffjobs.com/pl/job/junior-data-sci...
9,2020-10-24,Onwelo S.A.,Junior Fullstack Developer,Junior,https://nofluffjobs.com,https://nofluffjobs.com/pl/job/junior-fullstac...


In [12]:
df.columns

Index(['publication_date', 'company', 'title', 'position', 'website',
       'link_url'],
      dtype='object')